In [ ]:
import warnings
warnings.filterwarnings("ignore")

import tensorflow as tf
import os
import numpy as np
import random
from tqdm.auto import tqdm
from skimage.io import imread, imshow
from skimage.transform import resize
import matplotlib.pyplot as plt
from glob import glob
import cv2
from skimage.io import imread, imshow, show

from tensorflow import keras
from keras import layers
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input

seed = 42
np.random.seed = seed  # To get same random seed everytime we run the whole thing.
tf.random.set_seed(42)
random.seed(seed)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Importing The Dataset Paths

In [ ]:
print(len(os.listdir("/content/drive/MyDrive/ICCIT 2023/Selected_data/H1")))
print(len(os.listdir("/content/drive/MyDrive/ICCIT 2023/Selected_data/H2")))
print(len(os.listdir("/content/drive/MyDrive/ICCIT 2023/Selected_data/H3")))
print(len(os.listdir("/content/drive/MyDrive/ICCIT 2023/Selected_data/H5")))
print(len(os.listdir("/content/drive/MyDrive/ICCIT 2023/Selected_data/H6")))

700
700
700
700
700


In [ ]:
!pip install --root-user-action=ignore split-folders

In [ ]:
# Only one time run cell
import splitfolders

input_folder = "/content/drive/MyDrive/ICCIT 2023/Selected_data"
output = "/content/split_data/"

splitfolders.ratio(input_folder, output=output, seed=42, ratio=(0.90, 0.10))

Copying files: 3500 files [00:45, 77.46 files/s] 


## Model Creation

In [ ]:
learning_rate = 0.001
weight_decay = 0.0001
batch_size = 64
num_epochs = 100
image_size = 72  # We'll resize input images to this size
patch_size = 6  # Size of the patches to be extract from the input images
num_patches = (image_size // patch_size) ** 2
projection_dim = 64
num_heads = 4
transformer_units = [
    projection_dim * 2,
    projection_dim,
]  # Size of the transformer layers
transformer_layers = 4
mlp_head_units = [2048, 1024]  # Size of the dense layers of the final classifier

## Data Augmentation

In [ ]:
data_augmentation = keras.Sequential(
    [
        layers.Normalization(),
        layers.Resizing(image_size, image_size),
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(factor=0.02),
        layers.RandomZoom(height_factor=0.2, width_factor=0.2),
    ],
    name="data_augmentation",
)

# Compute the mean and the variance of the training data for normalization.
# data_augmentation.layers[0].adapt(x_train)

## MLP Layer

In [ ]:
def mlp(x, hidden_units, dropout_rate):
    for units in hidden_units:
        x = layers.Dense(units, activation=tf.nn.gelu)(x)
        x = layers.Dropout(dropout_rate)(x)
    return x

## Patch Creation as a layer

In [ ]:
class Patches(layers.Layer):
    def __init__(self, patch_size):
        super().__init__()
        self.patch_size = patch_size

    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images=images,
            sizes=[1, self.patch_size, self.patch_size, 1],
            strides=[1, self.patch_size, self.patch_size, 1],
            rates=[1, 1, 1, 1],
            padding="VALID",
        )
        patch_dims = patches.shape[-1]
        patches = tf.reshape(patches, [batch_size, -1, patch_dims])
        return patches

## Implementing the Patch Encoding Layer

In [ ]:
class PatchEncoder(layers.Layer):
    def __init__(self, num_patches, projection_dim):
        super().__init__()
        self.num_patches = num_patches
        self.projection = layers.Dense(units=projection_dim)
        self.position_embedding = layers.Embedding(
            input_dim=num_patches, output_dim=projection_dim
        )

    def call(self, patch):
        positions = tf.range(start=0, limit=self.num_patches, delta=1)
        encoded = self.projection(patch) + self.position_embedding(positions)
        return encoded

## ViT Model

In [ ]:
def create_vit_classifier(layer):

    input_shape = (256, 256, 3)
    num_classes = 5

    inputs = layers.Input(shape=input_shape)
    # Augment data.
    augmented = layer(inputs)
    # Create patches.
    patches = Patches(patch_size)(augmented)
    # Encode patches.
    encoded_patches = PatchEncoder(num_patches, projection_dim)(patches)

    # Create multiple layers of the Transformer block.
    for _ in range(transformer_layers):
        # Layer normalization 1.
        x1 = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
        # Create a multi-head attention layer.
        attention_output = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=projection_dim, dropout=0.1
        )(x1, x1)
        # Skip connection 1.
        x2 = layers.Add()([attention_output, encoded_patches])
        # Layer normalization 2.
        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
        # MLP.
        x3 = mlp(x3, hidden_units=transformer_units, dropout_rate=0.1)
        # Skip connection 2.
        encoded_patches = layers.Add()([x3, x2])

    # Create a [batch_size, projection_dim] tensor.
    representation = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
    representation = layers.Flatten()(representation)
    representation = layers.Dropout(0.5)(representation)
    # Add MLP.
    features = mlp(representation, hidden_units=mlp_head_units, dropout_rate=0.5)
    # Classify outputs.
    logits = layers.Dense(num_classes)(features)
    # Create the Keras model.
    model = keras.Model(inputs=inputs, outputs=logits)

    return model

model = create_vit_classifier(data_augmentation)
model.summary()


## Run, Train, Test

In [ ]:
# from sklearn.model_selection import StratifiedKFold
# cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

fold_no = 1
acc_per_fold = []
history_all = []

for k in range(3):

    print('   ')
    print(f'Training for fold {fold_no} ...', '\n')
    tf.random.set_seed(42+k)

    data_generator_with_aug = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   horizontal_flip=True,
                                   vertical_flip=True)

    data_generator_no_aug = ImageDataGenerator(preprocessing_function=preprocess_input)

    train_generator = data_generator_with_aug.flow_from_directory(
                        '/content/split_data/train',
                        target_size=(256, 256),
                        batch_size=batch_size,
                        class_mode='categorical',
                        shuffle=True)

    validation_generator = data_generator_no_aug.flow_from_directory(
                        '/content/split_data/val',
                        target_size=(256, 256),
                        batch_size=batch_size,
                        class_mode='categorical',
                        shuffle=True)




    data_augmentation = keras.Sequential([
                    layers.Normalization(),
                    layers.Resizing(image_size, image_size),
                    # layers.RandomFlip("horizontal"),
                    layers.RandomRotation(factor=0.02),
                    layers.RandomZoom(height_factor=0.1, width_factor=0.1)],
                                         name="data_augmentation")


    model = create_vit_classifier(data_augmentation)

    # optimizer = tfa.optimizers.AdamW(learning_rate=learning_rate,
    #                                  weight_decay=weight_decay)

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                  metrics=[tf.keras.metrics.CategoricalAccuracy(name="accuracy")])

    history = model.fit(
        train_generator,
        epochs=num_epochs,
        validation_data = validation_generator,
        verbose = 1)

    result = model.evaluate(validation_generator)
    print(f"Test accuracy: {round(result[1] * 100, 2)}%", '\n')

    acc_per_fold.append(result[1]*100)
    history_all.append(history)

    fold_no = fold_no + 1


   
Training for fold 1 ... 

Found 3150 images belonging to 5 classes.
Found 350 images belonging to 5 classes.
Epoch 1/100
50/50 [==============================] - 32s 214ms/step - loss: 4.8430 - accuracy: 0.3502 - val_loss: 1.5855 - val_accuracy: 0.4571
Epoch 2/100
50/50 [==============================] - 10s 198ms/step - loss: 1.6302 - accuracy: 0.3879 - val_loss: 1.3170 - val_accuracy: 0.5000
Epoch 3/100
50/50 [==============================] - 10s 198ms/step - loss: 1.4144 - accuracy: 0.4321 - val_loss: 1.1898 - val_accuracy: 0.5143
Epoch 4/100
50/50 [==============================] - 10s 198ms/step - loss: 1.3323 - accuracy: 0.4470 - val_loss: 1.1898 - val_accuracy: 0.4686
Epoch 5/100
50/50 [==============================] - 10s 200ms/step - loss: 1.2585 - accuracy: 0.4644 - val_loss: 1.1162 - val_accuracy: 0.5086
Epoch 6/100
50/50 [==============================] - 10s 199ms/step - loss: 1.2144 - accuracy: 0.4990 - val_loss: 1.0523 - val_accuracy: 0.5714
Epoch 7/100
50/50 [====